In [1]:
# import laion_clap
# from torchsummary import summary
import pandas as pd
# import swifter
import dask.dataframe as dd
from dask.multiprocessing import get

In [24]:
import torch
check = torch.load('630k-audioset-best.pt')
check.keys()

dict_keys(['epoch', 'name', 'state_dict', 'optimizer'])

In [2]:
import subprocess
import os
from pathlib import Path

def download_clip(
    video_identifier,
    output_filename,
    start_time,
    num_attempts=1,
    url_base='https://www.youtube.com/watch?v='
):
    status = False

    command = f"""yt-dlp -x --audio-format wav -f bestaudio -o "{output_filename}" --download-sections "*{start_time}-{start_time+10}" --force-keyframes-at-cuts --ffmpeg-location "C:\\Users\\3924s\\OneDrive\\Desktop\\Spring 2023\\ECE 285 Deep gen\\Project\\ffmpeg-master-latest-win64-gpl\\ffmpeg-master-latest-win64-gpl\\bin" {url_base}{video_identifier}""".strip()
    use_command = command

    attempts = 0
    while True:
        try:
            output = subprocess.check_output(use_command, shell=True,
                                                stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            attempts += 1
            print("Failed", output_filename)
            if attempts == num_attempts:
                return status, err.output
        else:
            break

    # Check if the video was successfully saved.
    status = os.path.exists(output_filename)
    return status, 'Downloaded'

def process(example):
    outfile_path = str(data_dir / f"{example['audiocap_id']}.wav")
    status = True
    if not os.path.exists(outfile_path):
        status = False
        status, log = download_clip(
            example['youtube_id'],
            outfile_path,
            example['start_time'],
        )
    example['caption'] = example['caption'].replace('\n', ' ')
    example['audio'] = outfile_path
    example['download_status'] = status
    return example


In [3]:
data = pd.read_csv(r'dataset\train_remaining.csv')
data.head()

,audiocap_id,youtube_id,start_time,caption
0,270,-5weqk40jaI,17,Strong wind against the microphone simultaneou...
1,19398,77fnehVF7wE,1,A dog barks twice and then whimpers
2,91139,r1nicOVtvkQ,130,A woman talks nearby as water pours
3,58146,UDGBjjwyaqE,20,Multiple clanging and clanking sounds
4,11542,3eJ9RynJzP8,80,"The wind is blowing, insects are singing, and ..."


In [4]:
exist_data = data[20000:]

In [5]:
print(len(exist_data))
exist_data.head()

22320


,audiocap_id,youtube_id,start_time,caption
20000,28359,BWAqer2fVY0,30,A woman talks nearby at moderate volume
20001,11111,3ThQo3IhY-A,30,"Motor vehicle engines are running, and people ..."
20002,11110,3Te0zSY6KSE,220,A man mumbling while crumpling something
20003,11113,3Tlc949kOMM,10,A motorcycle is revved and people talk
20004,11112,3Tkefu0HHbA,30,A locomotive is passing by and blows its whistle


In [6]:
data_dir = "./audio_data_20000_end" # Where to save the data

# Just select some samples 
# small_data = data[:samples_to_load]
# Create directory where data will be saved
data_dir = Path(data_dir)
data_dir.mkdir(exist_ok=True, parents=True)

ddata = dd.from_pandas(exist_data, npartitions=30)

music_data = ddata.map_partitions(lambda df: df.apply(process, axis=1)).compute(scheduler='processes')

Failed audio_data_20000_end\1.wav
Failed audio_data_20000_end\1.wav


In [17]:
print(music_data.shape)
music_data.head()

(2475, 6)


,audiocap_id,youtube_id,start_time,caption,audio,download_status
0,97151,vfY_TJq7n_U,130,"Rustling occurs, ducks quack and water splashe...",music_data_val\97151.wav,True
1,108945,tdWhHV3X25Q,60,An audience gives applause as a man yells and ...,music_data_val\108945.wav,True
2,107898,tw76HGONaKg,570,A man speaks over intermittent keyboard taps,music_data_val\107898.wav,True
3,107893,y2bVZ7rz-5M,280,Motor noise is followed by a horn honking and ...,music_data_val\107893.wav,True
4,107892,ti66RjZWTp0,20,A male speaks as metal clicks and a gun fires ...,music_data_val\107892.wav,False
